# <\Collecting Data>

### JobTitle, PostDate, JobLink, Number of Applicants, CompanyName, Region

In [ ]:
# Using Selenium to search on Linkedin dynamically with the city
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

print('Enter the region:')
region = input()

driver = webdriver.Chrome("C:/Users/ /chromedriver_win32/chromedriver.exe")
driver.get("https://www.linkedin.com/jobs/")
search_box = driver.find_element_by_name("location").clear()
search_box = driver.find_element_by_name("location")


search_box.send_keys(region)
search_box.send_keys(Keys.ENTER)
driver.minimize_window()
driver.maximize_window()


In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd




#url= driver.current_url
url="https://www.linkedin.com/jobs/search/?geoId=101591388&location=Kelowna%2C%20British%20Columbia%2C%20Canada"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
jobs = requests.get(url, headers=headers)
status=jobs.status_code
jobs = bs(jobs.content, 'html.parser')
print(status)


#Get the total numer of results
listSummary=[item.get_text() for item in jobs.find_all('label',class_='filter-list__list-item-label')]
for i in range(0,len(listSummary)):
    
    if re.match('Any Time',listSummary[i]):
        resultsTot=int(re.findall('\d+', re.sub(",", "", listSummary[i]))[0])
        print("Total Jobs:", resultsTot)
        

count=0
listtitle=[]     #-- Job Title 
listdate=[]      #-- Job Date 
listlink=[]      #-- Job link (details) 
#--- new code 
listappl=[]      #-- Number of Applicants

listcompany=[]   #List of companies names
listregion=[]    #List of Region


while count<= resultsTot:     
    #-- Job Title 
    list1 = [item.get_text() for item in jobs.find_all('h3',class_='result-card__title job-result-card__title')]
    #list1 = [item.get_text() for item in jobs.find_all('h3',class_='result-card__title job-result-card__title job-result-tile__title')] 
    
    #-- Job Date Posted    
    list2 = [item.get('datetime') for item in jobs.find_all('time')] # ,class_='job-result-card__listdate')]
    
    #-- Job link      
    list3 = [item.get('href') for item in jobs.find_all('a')] #-- use Regex here: https://ca.linkedin.com/jobs/view/
    
    #-- Job company
    list4 = [item.get_text() for item in jobs.find_all('h4',class_='result-card__subtitle job-result-card__subtitle')]
    
    #-- Job region
    list5 = [item.get_text() for item in jobs.find_all('span',class_='job-result-card__location')]
    
    count=count+25
    
    
    for i in range(0,len(list1)):
        listtitle.append(list1[i])
        listdate.append(list2[i])        

    for i in range (0,len(list3)):
        x=list3[i][0:34]
        if re.match(r'https://ca.linkedin.com/jobs/view/', x) != None:
            listlink.append(list3[i]) 
            urldesc=list3[i]
            
            urldescF='https://www.linkedin.com/jobs/view/'+urldesc[34:]#+'&originalSubdomain=ca'
            jobsd = requests.get(urldescF, headers=headers)
            status=jobsd.status_code
            jobsd = bs(jobsd.content, 'html.parser')               
   
            #--- New code: updated 
            numAplicants=[itemx.get_text() for itemx in jobsd.find_all('span', class_='topcard__flavor--metadata topcard__flavor--bullet num-applicants__caption')]
            if len(numAplicants) > 0: 
                nApplicants=numAplicants[0]
            else:
                nApplicants='Less than 25 applicants'
            
            listappl.append(nApplicants) 
            #--- Finish new code 
            
        
    for i in range(0, len(list4)):
        listcompany.append(list4[i])
        
    for i in range(0, len(list5)):
        listregion.append(list5[i])

    print('1', len(listtitle))
    #print('2', len(listdate))
    #print('3', len(listlink))
    #print('4', len(listappl))
    #print('5', len(listcompany))
    #print('6', len(listregion))
    
    #url= driver.current_url
    url="https://www.linkedin.com/jobs/search/?geoId=101591388&location=Kelowna%2C%20British%20Columbia%2C%20Canada"+"&start="+str(count)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
    jobs = requests.get(url, headers=headers)
    jobs = bs(jobs.content, 'html.parser')
    
    
    if count == 50:
        break

200
Total Jobs: 1463
1 25
1 50


### Data into DataFrame

In [2]:
dataframe=pd.DataFrame()
dataframe['JobTitle']  = listtitle
dataframe['JobDate'] = listdate 
dataframe['JobLink'] = listlink
dataframe['Applicants'] = listappl 
dataframe['Company'] = listcompany
dataframe['Region'] = listregion

In [3]:
dataframe.head(20) 

,JobTitle,JobDate,JobLink,Applicants,Company,Region
0,Remote Data Entry Clerk - Work at Home,2020-07-24,https://ca.linkedin.com/jobs/view/remote-data-...,40 applicants,UsaSurveyJobBoard,"Kelowna, British Columbia, Canada"
1,Warehouse & Inventory Administrator,2020-07-26,https://ca.linkedin.com/jobs/view/warehouse-in...,Less than 25 applicants,Sun-Rype Products Ltd.,"Kelowna, British Columbia, Canada"
2,Research Coordinator,2020-07-23,https://ca.linkedin.com/jobs/view/research-coo...,Less than 25 applicants,The University of British Columbia,"Kelowna, British Columbia, Canada"
3,Client Advisor Fall Student 2020 Okanagan,2020-07-24,https://ca.linkedin.com/jobs/view/client-advis...,Less than 25 applicants,RBC,"Kelowna, British Columbia, Canada"
4,"Producer, Radio (English Services)",2020-07-21,https://ca.linkedin.com/jobs/view/producer-rad...,Less than 25 applicants,CBC/Radio-Canada,"Kelowna, British Columbia, Canada"
5,Executive Assistant Office Manager,2020-07-27,https://ca.linkedin.com/jobs/view/executive-as...,Less than 25 applicants,Turner Group of Companies,"Kelowna, British Columbia, Canada"
6,Chief Executive Officer,2020-06-20,https://ca.linkedin.com/jobs/view/chief-execut...,Less than 25 applicants,Refresh Financial,"Kelowna, British Columbia, Canada"
7,Project Coordinator,2020-07-01,https://ca.linkedin.com/jobs/view/project-coor...,103 applicants,Aecon Group Inc.,"Kelowna, British Columbia, Canada"
8,NowMedia Content Strategist,2020-05-16,https://ca.linkedin.com/jobs/view/nowmedia-con...,Less than 25 applicants,NowMedia,"Kelowna, British Columbia, Canada"
9,Content Marketing Coordinator,2020-07-08,https://ca.linkedin.com/jobs/view/content-mark...,38 applicants,SKYTRAC,"Kelowna, British Columbia, Canada"


### DataFrame onto MongoDB

In [4]:
from pymongo import MongoClient
from bson.objectid import ObjectId

client=MongoClient('localhost', 27017)
db=client.jobs
col=db.July_26_new
#col.drop()

In [5]:
#DataFrame -> Dictionary
dataframe_dict = dataframe.to_dict('records')
col.insert_many(dataframe_dict)

### Adding job description onto MongoDB

In [6]:
#Adding Empty field for job description
col.update_many({},{'$set':{'job_description': ''}})

In [7]:
cur = col.find({},{'JobLink':1})
cur2 = col.find({},{'JobLink':1})
count=1

#while count <= col.count_documents({}):
while count <= 50:
      
    res = requests.get(cur.next()['JobLink'])
    res = bs(res.content, 'html.parser')
    
    description1 = [duty.get_text() for duty in res.find_all('ul')]
    print(description1)
    
    description2 = [duty.get_text() for duty in res.find_all('p')]
    print(description2)
        
    #description3 = [duty.get_text() for duty in res.find_all('strong')]
    #print(description3)
    
    #description4 = [duty.get_text() for duty in res.find_all('li')]
    #print(description4)
    
    col.update_one({'_id':cur2.next()['_id']}, {'$set':{'job_description':description1+description2}})
    
    count = count+1

['JobsPeopleLearning', '', '', 'Report this job', ' Earn money taking polls. Up to $35 per completed survey! Different payment methods, including Paypal, straight check, or on-line digital gift card codes Part Time Work remotely and earn additional income at home.', 'Seniority levelEntry levelEmployment typePart-timeJob functionAdministrativeIndustriesRetailConsumer ServicesFinancial Services', 'NLP Engineer – Chatbot (Remote - Part Time))NLP Engineer – Chatbot (Remote - Part Time))PredictivEye Inc.India4 days agoScientific Writer (Work from home)Scientific Writer (Work from home)Cactus CommunicationsIndia1 week agoApply NowMarketing SpecialistMarketing SpecialistAston Home Care LimitedStone, England, United Kingdom4 days agoSenior Financial AnalystSenior Financial AnalystPurolator Inc.Mississauga, Ontario, Canada3 days agoAdvisory AnalystAdvisory AnalystHatchMississauga, Ontario, Canada4 days agoGlobal Mobility/Immigration AssistantGlobal Mobility/Immigration AssistantWorkforce LogiqP

SSLError: HTTPSConnectionPool(host='ca.linkedin.com', port=443): Max retries exceeded with url: /jobs/view/work-from-home-job-administration-assistant-at-ers-work-options-1940916607?refId=10e685c2-1151-4651-a824-430bccc28e5d&position=2&pageNum=0&trk=public_jobs_job-result-card_result-card_full-click (Caused by SSLError(SSLError("bad handshake: SysCallError(10060, 'WSAETIMEDOUT')")))

In [ ]:
#col.update_many({}, {'$unset':{'job_description':description1+description2}})

# <\Matching keywords to Data>

### Get Keywords

In [8]:
# pip install pillow
from PIL import Image, ImageTk
import tkinter as tk
root = tk.Tk()
root.title('JobFinder')
canvas = tk.Canvas(root, height = 750, width = 1200)
canvas.pack()

def test_func(entry):
    global entrystr
    print('keywords :',entry)
    entrystr = str(entry)
    print(type(entrystr))
    print(entrystr)
    #label['text'] = job_summary()


#bg_image = tk.PhotoImage(file='bgimg1.png')
bg_label = tk.Label(root)
bg_label.place(x=0,y=0,relwidth=1,relheight=1)


frame = tk.Frame(root, bg = '#1e29fc', bd= 5)
frame.place(relx=0.5,rely=0.1,relwidth = 0.75, relheight = 0.1, anchor='n')

#label = tk.Label(frame, text ="enter your keywords")
#label.pack(side = 'left')
#label.grid(row =30 , column = 5)
#label.place(relx=0.3,rely=0,relwidth = 0.45, relheight = 0.25)


entry = tk.Entry(frame, font = 50)
#entry.pack(side = 'left')
#entry.grid(row =30 , column = 6)
entry.place(relwidth = 0.65, relheight = 1)


button = tk.Button(frame, text ="search", font = 50, command = lambda: [test_func(entry.get()),root.destroy()])
#button.pack(side = 'left')
#button.grid(row =30 , column = 7)
button.place(relx=0.7,relheight=1, relwidth = 0.3)

#lower_frame = tk.Frame(root, bg='#1e29fc', bd=10)
#lower_frame.place(relx=0.5, rely=0.25, relwidth=0.75, relheight=0.6, anchor='n')

#label = tk.Label(lower_frame)
#label.place(relwidth=1, relheight=1)

#exit_button = tk.Button(root, text='Exit',width=20, command=root.destroy)
#exit_button.pack()

root.mainloop()

keywords : project manager sql
<class 'str'>
project manager sql


In [9]:
def JobTitleMatch():
    entrylist=entrystr.split(' ')
    

    idlist=[]
    for i in range(0, len(entrylist)):
        for data in col.find({'JobTitle':{'$regex': entrylist[i], '$options':'i'}}, {'_id': 1, 'JobTitle':1})[:]:
            idlist.append(str(data['_id']))
            #print(data['_id'], data['JobTitle'])
        
            #count=col.count_documents({'JobTitle':{'$regex': entrylist[i], '$options':'i'}}
        
        
    #type(idlist[0]) #string
    #idlist
    
    idlist_unique = [idlist[0]]

    for j in range(1, len(idlist)):
        k=0
        while k < len(idlist_unique):
            if idlist[j] != idlist_unique[k]:
                if k < len(idlist_unique)-1:
                    k=k+1
                else:
                    idlist_unique.append(idlist[j])
            else:
                break

    print('Matched: ',len(idlist_unique))
    #idlist_unique
    
    
    global col2
    col2=db.July_26_new_unique

    for l in range(0, len(idlist_unique)):
        for unique_data in col.find({'_id':ObjectId(idlist_unique[l])}, {'_id':0, 'job_description':0}):
            col2.insert_one(unique_data)
            
    
    
    dataframe_match1=pd.DataFrame(list(col2.find()))
    JobTitleMatching = dataframe_match1.drop('_id', axis=1)
    return JobTitleMatching

### Keywords in JobTitle

In [ ]:
'''
idlist=[]
for i in range(0, len(entrylist)):
    for data in col.find({'JobTitle':{'$regex': entrylist[i], '$options':'i'}}, {'_id': 1, 'JobTitle':1})[:]:
        idlist.append(str(data['_id']))
        print(data['_id'], data['JobTitle'])
        
        #count=col.count_documents({'JobTitle':{'$regex': entrylist[i], '$options':'i'}}
        
        
#type(idlist[0]) #string
idlist
'''

In [ ]:
'''
idlist_unique = [idlist[0]]

for j in range(1, len(idlist)):
    k=0
    while k < len(idlist_unique):
        if idlist[j] != idlist_unique[k]:
            if k < len(idlist_unique)-1:
                k=k+1
            else:
                idlist_unique.append(idlist[j])
        else:
            break

print('Matched: ',len(idlist_unique))
idlist_unique
'''

In [ ]:

'''
from bson.objectid import ObjectId

col2=db.July_26_second_unique

for l in range(0, len(idlist_unique)):
    for unique_data in col.find({'_id':ObjectId(idlist_unique[l])}, {'_id':0, 'job_description':0}):
        col2.insert_one(unique_data)
'''



In [ ]:
'''
dataframe_match1=pd.DataFrame(list(col2.find()))
JobTitleMatching = dataframe_match1.drop('_id', axis=1)
JobTitleMatching
'''


### Keywords in JobDescription

In [10]:
#url= driver.current_url
url="https://www.linkedin.com/jobs/search/?geoId=101591388&location=Kelowna%2C%20British%20Columbia%2C%20Canada"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
jobs = requests.get(url, headers=headers)
status=jobs.status_code
jobs = bs(jobs.content, 'html.parser')
print(status)

count = 0
listmatch=[]     #-- number of words matched
listwords=[]     #-- words matched  

entrylist=entrystr.split(' ')


while count<= resultsTot:     
    
    count=count+25


    for i in range (0,len(list3)):
        x=list3[i][0:34]
        if re.match(r'https://ca.linkedin.com/jobs/view/', x) != None:
            
            urldesc=list3[i]
            
            urldescF='https://www.linkedin.com/jobs/view/'+urldesc[34:]#+'&originalSubdomain=ca'
            jobsd = requests.get(urldescF, headers=headers)
            status=jobsd.status_code
            jobsd = bs(jobsd.content, 'html.parser')               
   
                
            listDesc=[item.get_text() for item in jobsd.find_all('ul')]+[item.get_text() for item in jobsd.find_all('p')] 
            
            
            #Kwords = ['PROJECT','LEADERSHIP','PMP','PMI-ACP','COMMUNICATION','PORTUGUESE','ERP','ORACLE'] 
            KwordCount=0 
            #print (urldescF)
            kmatches=''
            #for w in Kwords:
            for w in entrylist:
                  
                myre = re.compile(w.upper())
        
                ix=0 
                Kwordfound=0
                while Kwordfound==0 and ix < len(listDesc):
                    #print (listDesc[ix].upper().split('REPORT THIS JOB'))
                    #print(myre.findall(listDesc[ix].upper()))
                    #sleep(1)                    
                    if (len(myre.findall(listDesc[ix].upper())) > 0):
                        KwordCount=KwordCount+1
                        Kwordfound=1
                        kmatches=kmatches+', '+w

                    ix=ix+1
      
            listmatch.append(KwordCount)   
            #-- new code 
            listwords.append(kmatches)   
            #-- 
         
    print('1', len(listmatch))
    print('2', len(listwords))
    
    
    
    #url= driver.current_url
    url="https://www.linkedin.com/jobs/search/?geoId=101591388&location=Kelowna%2C%20British%20Columbia%2C%20Canada"+"&start="+str(count)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}
    jobs = requests.get(url, headers=headers)
    jobs = bs(jobs.content, 'html.parser')
    
    if count == 50:
        break


200
1 25
2 25
1 50
2 50


In [11]:
dataframe_match2=pd.DataFrame()
dataframe_match2['Num_Matches']  = listmatch
dataframe_match2['Words Matched'] = listwords
dataframe_match2['JobTitle']  = listtitle
dataframe_match2['JobDate'] = listdate 
dataframe_match2['JobLink'] = listlink
dataframe_match2['Applicants'] = listappl 
dataframe_match2['Company'] = listcompany
dataframe_match2['Region'] = listregion

In [12]:
JobDescriptionMatching = dataframe_match2.sort_values(by='Num_Matches', ascending=False)
JobDescriptionMatching.head(20)

,Num_Matches,Words Matched,JobTitle,JobDate,JobLink,Applicants,Company,Region
49,2,", project, manager",Educator,2020-07-21,https://ca.linkedin.com/jobs/view/educator-at-...,Less than 25 applicants,lululemon,"Kelowna, British Columbia, Canada"
43,2,", project, manager",Student Recruiter,2020-07-03,https://ca.linkedin.com/jobs/view/student-recr...,Less than 25 applicants,Canadian Blood Services,"Kelowna, British Columbia, Canada"
23,2,", project, manager",Financial Analyst,2020-07-19,https://ca.linkedin.com/jobs/view/financial-an...,131 applicants,TNI The Network Inc.,"Peachland, British Columbia, Canada"
24,2,", project, manager",Educator,2020-07-21,https://ca.linkedin.com/jobs/view/educator-at-...,Less than 25 applicants,lululemon,"Kelowna, British Columbia, Canada"
46,2,", project, manager",Project Manager,2020-07-26,https://ca.linkedin.com/jobs/view/project-mana...,Less than 25 applicants,Interior Health,"Kelowna, British Columbia, Canada"
26,2,", project, manager",Warehouse & Inventory Administrator,2020-07-26,https://ca.linkedin.com/jobs/view/warehouse-in...,Less than 25 applicants,Sun-Rype Products Ltd.,"Kelowna, British Columbia, Canada"
27,2,", project, manager",Research Coordinator,2020-07-23,https://ca.linkedin.com/jobs/view/research-coo...,Less than 25 applicants,The University of British Columbia,"Kelowna, British Columbia, Canada"
45,2,", project, manager",User Support Technician,2020-07-09,https://ca.linkedin.com/jobs/view/user-support...,63 applicants,Statistics Canada,"Kelowna, British Columbia, Canada"
44,2,", project, manager",Project Coordinator,2020-05-10,https://ca.linkedin.com/jobs/view/project-coor...,74 applicants,WESCO Distribution,"Kelowna, British Columbia, Canada"
30,2,", project, manager",Executive Assistant Office Manager,2020-07-27,https://ca.linkedin.com/jobs/view/executive-as...,Less than 25 applicants,Turner Group of Companies,"Kelowna, British Columbia, Canada"


In [ ]:
'''
def job_summary():
    
    jobnames = JobDescriptionMatching['JobTitle']
    #print(jobnames)
    jobsite = JobDescriptionMatching['JobLink']
    #print(jobsite)
    jobcompany = JobDescriptionMatching['Company']
    #print(jobcompany)
    global finaldf
    finaldf = pd.DataFrame()
    finaldf['JobNames'] = jobnames
    finaldf['JobCompany'] = jobcompany
    finaldf['JobSite'] = jobsite
    
    return finaldf
job_summary()
'''

In [13]:
# pip install pillow
from PIL import Image, ImageTk
import tkinter as tk
from pandastable import Table, TableModel

root = tk.Tk()
root.title('JobFinder')
canvas = tk.Canvas(root, height = 750, width = 1200)
canvas.pack()


def test_func(entry):
    global entrystr
    print('keywords :',entry)
    entrystr = str(entry)
    print(type(entrystr))
    print(entrystr)
    label['text'] = JobTitleMatch()

def dropmongo():
    col2.drop()

bg_image = tk.PhotoImage(file='bgimg1.png')
bg_label = tk.Label(root, image = bg_image)
bg_label.place(x=0,y=0,relwidth=1,relheight=1)


frame = tk.Frame(root, bg = '#0a4761', bd= 5)
frame.place(relx=0.5,rely=0.1,relwidth = 0.75, relheight = 0.1, anchor='n')

#label = tk.Label(frame, text ="enter your keywords")
#label.pack(side = 'left')
#label.grid(row =30 , column = 5)
#label.place(relx=0.3,rely=0,relwidth = 0.45, relheight = 0.25)


entry = tk.Entry(frame, font = 50)
#entry.pack(side = 'left')
#entry.grid(row =30 , column = 6)
entry.place(relwidth = 0.65, relheight = 1)


button = tk.Button(frame, text ="search", font = 50, command = lambda: [test_func(entry.get()), dropmongo()])
#button.pack(side = 'left')
#button.grid(row =30 , column = 7)
button.place(relx=0.7,relheight=1, relwidth = 0.3)

lower_frame = tk.Frame(root, bg='#0a4761', bd=10)
lower_frame.place(relx=0.5, rely=0.25, relwidth=0.75, relheight=0.6, anchor='n')

label = tk.Label(lower_frame, anchor = "nw", justify = 'left')
label.place(relwidth=1, relheight=1)

df = JobDescriptionMatching[['JobTitle','Num_Matches','JobDate','JobLink','Applicants','Company','Region']]
pt = Table(lower_frame, dataframe=df,
                                    showtoolbar=True, showstatusbar=True)

pt.show()

exit_button = tk.Button(root, text='Exit',width=20, command=root.destroy)
exit_button.pack()



root.mainloop()
